In [1]:
import platform,os, sys
os_name = platform.system()

In [2]:
import findspark

In [3]:
if os_name == "Windows":
    findspark.init()
    img_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/deep-learning-images"
    temp_loc = "D:/temp"
    master = "local[*]"
    driver_memory = "24g"
else:
    spark_home = "/usr/hdp/current/spark2-client"
    findspark.init(spark_home)
    hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]
    img_dir = "/home/kranthidr/data/sdl-flowers/"
    temp_loc = os.getenv("HOME") + "/temp"
    master = "yarn"
    driver_memory = "2g"
    #sys.path.append(os.path.expanduser('hdfs:///user/kranthidr/pythonpath/sparkdl.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/sparkdl.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/tensorframes.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/tensorflow.zip'))
    sys.path.append(os.path.expanduser("~/.local/lib/python2.7/site-packages"))

In [4]:
sys.path

['C:\\Spark\\spark-hadoop\\python',
 'C:\\Spark\\spark-hadoop\\python\\lib\\py4j-0.10.6-src.zip',
 '',
 'd:\\learn\\.virtualenvs\\pyspark-lab\\scripts\\python36.zip',
 'd:\\learn\\.virtualenvs\\pyspark-lab\\DLLs',
 'd:\\learn\\.virtualenvs\\pyspark-lab\\lib',
 'd:\\learn\\.virtualenvs\\pyspark-lab\\scripts',
 'c:\\users\\kranthikumar\\anaconda3\\Lib',
 'c:\\users\\kranthikumar\\anaconda3\\DLLs',
 'd:\\learn\\.virtualenvs\\pyspark-lab',
 'd:\\learn\\.virtualenvs\\pyspark-lab\\lib\\site-packages',
 'D:\\Learn\\PythonProjects\\ml_lab\\pyspark-lab',
 'd:\\learn\\.virtualenvs\\pyspark-lab\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\KranthiKumar\\.ipython']

In [5]:
findspark.find()

'C:\\Spark\\spark-hadoop'

In [6]:
img_dir

'D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/deep-learning-images'

In [7]:
temp_loc

'D:/temp'

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master(master)\
.config("spark.driver.memory", driver_memory)\
.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11")\
.appName("ch31_DeepLearningTuning").getOrCreate()

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "8g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "10")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [9]:
sc = spark.sparkContext

In [10]:
for x in sc._conf.getAll():
    if "/proxy/" in x[1]:
        print(x[1])

In [11]:
spark

In [12]:
import keras.applications

Using TensorFlow backend.


In [13]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(weights= None, classes=2)
model.save(temp_loc + "/model-full-tmp.h5")

# Signature: InceptionV3(include_top=True, weights='imagenet', 
#                        input_tensor=None, input_shape=None, pooling=None, classes=1000)
# Docstring:
# Instantiates the Inception v3 architecture.

In [14]:
import os
#from os import listdir
def absoluteFilePaths(directory):
   for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))

In [15]:
absoluteFilePaths(img_dir + "/tulips")

<generator object absoluteFilePaths at 0x0000021F68DDF0F8>

In [16]:
tulips_files = [str(f) for f in absoluteFilePaths(img_dir + "/tulips")]  # make "local" file paths for images
daisy_files = [str(f) for f in absoluteFilePaths(img_dir + "/daisy")]  # make "local" file paths for images

In [17]:
tulips_files[:1]

['D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\tulips\\3498663243_42b39b4185_m.jpg']

In [18]:
daisy_files[:1]

['D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\daisy\\10555815624_dc211569b0.jpg']

In [19]:
import pyspark.ml.linalg as spla
import pyspark.sql.types as sptyp
import numpy as np

In [20]:
def CreateTrainImageUriandLabels(image_uris, label, cardinality):
  # Create image categorical labels (integer IDs)
  local_rows = []
  for uri in image_uris:
    label_inds = np.zeros(cardinality)
    label_inds[label] = 1.0
    one_hot_vec = spla.Vectors.dense(label_inds.tolist())
    _row_struct = {"uri": uri, "one_hot_label": one_hot_vec, "label": float(label)}
    row = sptyp.Row(**_row_struct)
    local_rows.append(row)

  image_uri_df = spark.createDataFrame(local_rows)
  return image_uri_df

In [21]:
label_cardinality = 2

In [22]:
tulips_uri_df = CreateTrainImageUriandLabels(tulips_files,1,label_cardinality)

In [23]:
daisy_uri_df = CreateTrainImageUriandLabels(daisy_files,0,label_cardinality)

In [24]:
tulips_uri_df.show(2,False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3498663243_42b39b4185_m.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3501996215_1c6d1a3386_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [25]:
daisy_uri_df.show(2, False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|0.0  |[1.0,0.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\daisy\10555815624_dc211569b0.jpg  |
|0.0  |[1.0,0.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\daisy\10555826524_423eb8bf71_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [26]:
#tulips_train, tulips_test, _ = tulips_uri_df.randomSplit([0.005, 0.005, 0.99]) 
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
tulips_train, tulips_test = tulips_uri_df.randomSplit([0.8, 0.2]) 

In [27]:
#daisy_train, daisy_test, _ = daisy_uri_df.randomSplit([0.005, 0.005, 0.99])    
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
daisy_train, daisy_test = daisy_uri_df.randomSplit([0.8, 0.2]) 

In [28]:
train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)

In [29]:
train_df.repartition(100).cache()
test_df.repartition(100).cache()

DataFrame[label: double, one_hot_label: vector, uri: string]

In [30]:
train_df.show(3, False)
test_df.show(3, False)

+-----+-------------+---------------------------------------------------------------------------------------------------------+
|label|one_hot_label|uri                                                                                                      |
+-----+-------------+---------------------------------------------------------------------------------------------------------+
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3498663243_42b39b4185_m.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3501996215_1c6d1a3386_n.jpg|
|1.0  |[0.0,1.0]    |D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\tulips\3502085373_edc2c36992_n.jpg|
+-----+-------------+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows

+-----+-------------+----------------------------------------------------------

In [31]:
import PIL.Image
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from sparkdl.estimators.keras_image_file_estimator import KerasImageFileEstimator

In [32]:
def load_image_from_uri(local_uri):
  img = (PIL.Image.open(local_uri).convert('RGB').resize((299, 299), PIL.Image.ANTIALIAS))
  img_arr = np.array(img).astype(np.float32)
  img_tnsr = preprocess_input(img_arr[np.newaxis, :])
  return img_tnsr

In [33]:
train_df.select("uri").first()[0]

'D:\\Learn\\GitRepos\\Spark-The-Definitive-Guide\\data\\deep-learning-images\\tulips\\3498663243_42b39b4185_m.jpg'

In [34]:
test_load = load_image_from_uri(train_df.select("uri").first()[0])

In [35]:
test_load.shape

(1, 299, 299, 3)

In [36]:
# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
# train_df = train_df.repartition(100)
# test_df = test_df.repartition(100)

In [37]:
train_df.groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|   24|
|  1.0|   25|
+-----+-----+



In [38]:
# estimatorT = KerasImageFileEstimator( inputCol="uri",
#                                      outputCol="prediction",
#                                      labelCol="one_hot_label",
#                                      imageLoader=load_image_from_uri,
#                                      kerasOptimizer='adam',
#                                      kerasLoss='categorical_crossentropy', # why categorical_crossentropy
#                                      modelFile=temp_loc+"/model-full-tmp.h5", # local file path for model
#                                      kerasFitParams = {"batch_size": 32, "verbose": 0}
#                                    ) 

#     transformers = estimator.fit(image_dataset)
# Init docstring:
# __init__(self, inputCol=None, outputCol=None, 
#          outputMode="vector", labelCol=None,
#          modelFile=None, imageLoader=None, 
#          kerasOptimizer=None, kerasLoss=None,
#          kerasFitParams=None)

In [39]:
# fittedT = estimatorT.fit(train_df)

In [40]:
estimator = KerasImageFileEstimator( inputCol="uri",
                                     outputCol="prediction",
                                     labelCol="one_hot_label",
                                     imageLoader=load_image_from_uri,
                                     kerasOptimizer='adam',
                                     kerasLoss='categorical_crossentropy', # why categorical_crossentropy
                                     modelFile=temp_loc+"/model-full-tmp.h5" # local file path for model
                                   ) 

In [41]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [42]:
paramGrid = (
  ParamGridBuilder()
  .addGrid(estimator.kerasFitParams, [
                                      {"batch_size": 64, "verbose": 0}])
  .build()
)

In [43]:
# paramGrid = (
#   ParamGridBuilder()
#   .addGrid(estimator.kerasFitParams, [{"batch_size": 32, "verbose": 0},
#                                       {"batch_size": 64, "verbose": 0}])
#   .build()
# )

In [44]:
bc = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label" )
cv = CrossValidator(estimator=estimator, estimatorParamMaps=paramGrid, evaluator=bc, numFolds=2)

In [45]:
cvModel = cv.fit(train_df)

# ~\AppData\Local\Temp\spark-5780b8a4-c5ad-43ba-b92a-dfde3f4a482c
# \userFiles-2c99e8b3-645f-4bf2-bfc6-069f994f8c2c\databricks_tensorframes-0.5.0-s_2.11.jar
# \tensorframes\core.py in _add_graph(graph, builder, use_file)
#      59     if use_file:
#      60         # TODO: remove the dir and honor the existing one
# ---> 61         d = tempfile.mkdtemp("tensorframes", dir="/tmp")
#      62         tf.train.write_graph(graph, d, "proto.pb", False)
#      63         fname = d + "/proto.pb"

# d:\learn\.virtualenvs\pyspark-lab\lib\tempfile.py in mkdtemp(suffix, prefix, dir)
#     366         file = _os.path.join(dir, prefix + name + suffix)
#     367         try:
# --> 368             _os.mkdir(file, 0o700)
#     369         except FileExistsError:
#     370             continue    # try again

# FileNotFoundError: [WinError 3] The system cannot find the path specified: '/tmp\\tmpj9rq2nowtensorframes'

d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


INFO:tensorflow:Froze 378 variables.
Converted 378 variables to const ops.
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/tmp\\tmpj9rq2nowtensorframes'

In [ ]:
# cvModel = cv.fit(train_df)

# #
# File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\keras\engine\training_utils.py", line 137, in standardize_input_data
#     str(data_shape))
# ValueError: Error when checking target: expected predictions to have shape (1000,) but got array with shape (2,)


In [ ]:
# from keras.layers import Activation, Dense, Flatten
# from keras.models import Sequential

# model = Sequential()
# model.add(Flatten(input_shape=(299, 299, 3)))
# model.add(Dense(2))
# model.add(Activation("softmax"))
# model.save(temp_loc + "/model-full-tmp1.h5")  # saves to the local filesystem

#DURING FIT inKerasImageFileEstimator

# Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
# : org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 37.0 failed 1 times, 
#             most recent failure: Lost task 0.0 in stage 37.0 (TID 529, localhost, executor driver): 
#                     org.apache.spark.api.python.PythonException: Traceback (most recent call last):
#   File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\client\session.py", line 1361, in _do_call
#     return fn(*args)
#   File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\client\session.py", line 1340, in _run_fn
#     target_list, status, run_metadata)
#   File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 516, in __exit__
#     c_api.TF_GetCode(self.status.status))
# tensorflow.python.framework.errors_impl.InternalError: Blas GEMM launch failed : a.shape=(32, 268203), 
#         b.shape=(268203, 2), m=32, n=2, k=268203
# 	 [[Node: dense_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, 
#                                      _device="/job:localhost/replica:0/task:0/device:GPU:0"]
#        (flatten_1/Reshape, dense_1/kernel/read)]]
# 	 [[Node: dense_1/BiasAdd/_49 = _Recv[client_terminated=false, 
#                                          recv_device="/job:localhost/replica:0/task:0/device:CPU:0", 
                                         
#                                          send_device="/job:localhost/replica:0/task:0/device:GPU:0", 
#                                          send_device_incarnation=1, 
#                                          tensor_name="edge_63_dense_1/BiasAdd", tensor_type=DT_FLOAT, 
#                                          _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]